In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

In [18]:
# глобальные переменные

file_path_data = r'C:\Users\kakorin_is\Desktop\JupiterNotebook_files\ЮФО для сверки с предв. расчетом ЮД 02.03.xlsx'
file_path_activities = r'C:\Users\kakorin_is\Desktop\JupiterNotebook_files\Активности всех дата начала с 20.11.2022 по сейчас (дату создания и завершения чекать, у с_з дату начала).xlsx'
file_path_data_other = r'C:\Users\kakorin_is\Desktop\JupiterNotebook_files\вся иная - чтобы выбрать дела Жежелева в тот период.xlsx'
file_path_to_user_id = r'C:\Users\kakorin_is\Desktop\Свод\юристы РОО.xlsx'

start = date(2022, 11, 20) #2022, 11, 2 брать предшествующий день дню, с которого начать
end = date(2022, 12, 3) # 2022, 12, 3 брать следующий день после дня, на который заканчиваем включительно

user_to_check = 100000023

In [3]:
data = pd.read_excel(file_path_data, index_col=None)
data_other = pd.read_excel(file_path_data_other, index_col=None)
activities = pd.read_excel(file_path_activities, index_col=None)

__В режиме демонстрации переведу ФИО ответственных в user_id:__

Прим. в рабочем варианте поиск идет по user_to_check = ФИО 

In [5]:
user_id_data = pd.read_excel(file_path_to_user_id, index_col=None)
user_id_data.columns

Index(['Ответственный', 'роо', 'id юриста'], dtype='object')

In [6]:
user_id_data['id юриста']

0     100000001
1     100000002
2     100000003
3     100000004
4     100000005
5     100000006
6     100000007
7     100000008
8     100000009
9     100000010
10    100000011
11    100000012
12    100000013
13    100000014
14    100000015
15    100000016
16    100000017
17    100000018
18    100000019
19    100000020
20    100000021
21    100000022
22    100000023
23    100000024
24    100000025
Name: id юриста, dtype: int64

In [7]:
names_list = user_id_data['Ответственный'].values
user_id_list = user_id_data['id юриста'].values

dict_id = dict(zip(names_list, user_id_list))

In [8]:
# Добавляю user_id во все три датафрейма и далее буду работать именно с этим полем для идентификации пользователя
data['user_id'] = data['Ответственный'].apply(lambda x: dict_id[x])
data_other['user_id'] = data_other['Ответственный'].apply(lambda x: dict_id[x])
activities['user_id'] = activities['Ответственный'].apply(lambda x: dict_id[x])

In [9]:
# смотрим сколько строк и колонок в выгрузке из БД по активностям пользователей в Системе
activities.shape

(6037, 28)

In [10]:
# какие колонки в файле
activities.columns

Index(['Мероприятие', 'Индикатор', 'Тип', 'Тип активности Outbound', 'Статус',
       'Начало', 'Фактическое окончание', 'Результат', 'Соотношение к ПЗ',
       'Завершена с просрочкой', 'Клиент (ФЛ)', 'КМБ', 'Комментарии',
       '№ договора', 'Судебное дело', 'Адрес',
       'Дней просрочки на момент создания', 'Ответственный по клиенту',
       'Центр сбора', 'Номер дела о банкротстве', 'Ответственный',
       'Дата создания', 'Дата передачи', 'Общий комментарий', 'Описание',
       'Иная судебная работа', 'Дата публичных торгов (принятие на баланс)',
       'user_id'],
      dtype='object')

In [11]:
usefull_cols = ['Судебное дело','Иная судебная работа', 'Тип', 'Статус','Начало', 'Дата создания',\
                'Фактическое окончание','Результат','user_id',  'Комментарии']

activities = activities.loc[:, usefull_cols]

activities.head(2)

,Судебное дело,Иная судебная работа,Тип,Статус,Начало,Дата создания,Фактическое окончание,Результат,user_id,Комментарии
0,1-3259773763,NaN,Судебное заседание,Запланировано,2023-03-27 11:00:39,2023-02-28 10:33:39,NaT,NaN,100000011,NaN
1,1-3139369223,NaN,Судебное заседание,Запланировано,2023-03-28 11:25:20,2023-02-15 16:47:20,NaT,NaN,100000011,NaN


Уберу все активности созданные после даты end, оставляю созданные в проверяемом периоде активности с датой начала за рамками проверяемого периода, чтобы не обрезать реальную активность сотрудника в ПО.

In [12]:
# Функция преобразования даты
def fix_date(some_date):
    some_date = some_date.date()
    
    return some_date

# # Специально делаю отдельный столбец к корректной формой даты, чтобы можно было потом сравнить
activities['Дата создания испр'] = activities['Дата создания'].apply(lambda x: fix_date(x))
activities['Начало испр'] = activities['Начало'].apply(lambda x: fix_date(x))

In [13]:
activities.loc[:, ['Дата создания', 'Дата создания испр', 'Начало', 'Начало испр']].head(5)

,Дата создания,Дата создания испр,Начало,Начало испр
0,2023-02-28 10:33:39,2023-02-28,2023-03-27 11:00:39,2023-03-27
1,2023-02-15 16:47:20,2023-02-15,2023-03-28 11:25:20,2023-03-28
2,2023-02-02 16:58:57,2023-02-02,2023-03-28 11:30:57,2023-03-28
3,2023-02-27 15:14:51,2023-02-27,2023-03-29 12:00:51,2023-03-29
4,2023-02-02 18:07:17,2023-02-02,2023-04-03 09:45:17,2023-04-03


__Теперь сделаю две таблицы - одна кол-во с/з по дням проверяемого периода (назначенные с/з) и вторая - кол-во созданных активностей в проверяемом периоде с разбивкой по дням и типам активностей:__

In [14]:
activities['Тип'].value_counts()

Судебное заседание                1385
Запрос на оплату госпошлины        874
Запрос копий документов            790
Запрос расчета задолженности       788
Передача дела в суд                775
Подготовка обращения в суд         773
Судебное заседание СД 3Л           583
Категория нестандартности           50
Поступил документ АД                11
Запрос на корректировку ИД           6
Получение КОД/досье от Бизнеса       1
Запрос на выдачу дубликата           1
Name: Тип, dtype: int64

In [19]:
part1 = activities.loc[(activities['Начало испр'] > start) & (activities['Начало испр'] < end)]
activities_for_court_hearings_table = part1.loc[(part1['user_id'] == user_to_check) & \
                                                ((part1['Тип'] == 'Судебное заседание') | \
                                                 (part1['Тип'] == 'Судебное заседание СД 3Л'))]

activities_for_court_hearings_table.shape

(30, 12)

In [20]:
activities_for_court_hearings_table.head(2)

,Судебное дело,Иная судебная работа,Тип,Статус,Начало,Дата создания,Фактическое окончание,Результат,user_id,Комментарии,Дата создания испр,Начало испр
5329,1-2912713081,NaN,Судебное заседание,Завершено,2022-12-01 17:45:18,2022-11-16 14:58:18,2022-12-01 18:00:00,Требование удовлетворено,100000023,NaN,2022-11-16,2022-12-01
5349,NaN,1-1DY2NC9,Судебное заседание СД 3Л,Завершено,2022-12-01 15:43:10,2022-12-22 15:43:10,2022-12-01 15:50:10,Завершено,100000023,NaN,2022-12-22,2022-12-01


In [21]:
activities_for_court_hearings_table['Тип'].value_counts()

Судебное заседание          21
Судебное заседание СД 3Л     9
Name: Тип, dtype: int64

In [22]:
# поскольку тут только активности с/з, то смело группирую 
court_hearings_table = activities_for_court_hearings_table.groupby('Начало испр').agg({'Статус': 'count'})
court_hearings_table.reset_index(inplace=True)
court_hearings_table

,Начало испр,Статус
0,2022-11-21,3
1,2022-11-22,6
2,2022-11-23,4
3,2022-11-24,2
4,2022-11-25,1
5,2022-11-28,5
6,2022-11-29,1
7,2022-11-30,6
8,2022-12-01,2


__Инф. о кол-ве назначенных заседаний в отчетном периоде есть, теперь делаем таблицу сколько и каких активностей было создано в исследуемом периоде:__

In [23]:
user_activities = activities.loc[(activities['user_id'] == user_to_check) & \
                                  (activities['Дата создания испр'] > start) & \
                                  (activities['Дата создания испр'] < end)]

                                                 
user_activities['Тип'].value_counts()

Передача дела в суд             32
Запрос на оплату госпошлины     30
Запрос расчета задолженности    28
Запрос копий документов         28
Подготовка обращения в суд      27
Судебное заседание СД 3Л        12
Судебное заседание               8
Name: Тип, dtype: int64

In [24]:
user_activities.shape

(165, 12)

In [25]:
# сохраню список активностей чтобы не вылетали ошибки, в случаях когда какой-то тип активностей не создавался
user_type_acts_list = user_activities['Тип'].unique()

In [26]:
#  c помощью горячего кодирования добавляю бинарные колонки по каждому значению Типа активности
data_dummies = pd.get_dummies(user_activities['Тип'])    
data_dummies.head(5)


,Запрос копий документов,Запрос на оплату госпошлины,Запрос расчета задолженности,Передача дела в суд,Подготовка обращения в суд,Судебное заседание,Судебное заседание СД 3Л
3001,0,0,0,0,0,1,0
3242,0,0,0,0,0,1,0
4065,0,0,0,0,0,1,0
4070,0,0,0,0,0,1,0
4196,0,0,0,0,0,0,1


In [27]:
# присоединяю по индексам  таблицу к основной
user_activities = pd.merge(user_activities, data_dummies, left_index=True, right_index=True, how='outer')  # это таблица
user_activities.head(3)


,Судебное дело,Иная судебная работа,Тип,Статус,Начало,Дата создания,Фактическое окончание,Результат,user_id,Комментарии,Дата создания испр,Начало испр,Запрос копий документов,Запрос на оплату госпошлины,Запрос расчета задолженности,Передача дела в суд,Подготовка обращения в суд,Судебное заседание,Судебное заседание СД 3Л
3001,1-3037742503,NaN,Судебное заседание,Завершено,2023-01-20 09:45:38,2022-11-24 16:39:41,2023-01-20 11:00:00,Требование удовлетворено,100000023,NaN,2022-11-24,2023-01-20,0,0,0,0,0,1,0
3242,1-3010635577,NaN,Судебное заседание,Завершено,2023-01-17 14:30:44,2022-11-28 10:11:44,2023-01-17 15:05:00,Требование удовлетворено,100000023,NaN,2022-11-28,2023-01-17,0,0,0,0,0,1,0
4065,1-3020580419,NaN,Судебное заседание,Завершено,2022-12-27 11:40:39,2022-11-30 11:32:39,2022-12-27 12:06:00,Удовлетворено полностью,100000023,NaN,2022-11-30,2022-12-27,0,0,0,0,0,1,0


In [28]:
#кол-во строк совпадает - ничего не потерялось
user_activities.shape

(165, 19)

In [29]:
user_type_acts_list

array(['Судебное заседание', 'Судебное заседание СД 3Л',
       'Передача дела в суд', 'Подготовка обращения в суд',
       'Запрос на оплату госпошлины', 'Запрос расчета задолженности',
       'Запрос копий документов'], dtype=object)

In [30]:
dict_for_group = {}

for i in user_type_acts_list:
    dict_for_group[i] = 'sum'
    
user_activities_grouped = user_activities.groupby('Дата создания испр').agg(dict_for_group)    



user_activities_grouped

,Судебное заседание,Судебное заседание СД 3Л,Передача дела в суд,Подготовка обращения в суд,Запрос на оплату госпошлины,Запрос расчета задолженности,Запрос копий документов
Дата создания испр,,,,,,,
2022-11-21,0,3,5,0,0,0,0
2022-11-22,3,5,0,0,0,0,0
2022-11-23,0,0,0,0,1,1,1
2022-11-24,1,0,9,9,9,9,9
2022-11-25,0,0,0,0,1,1,1
2022-11-28,1,0,3,3,3,3,3
2022-11-29,1,0,6,6,8,6,6
2022-11-30,2,4,4,4,4,4,4
2022-12-02,0,0,5,5,4,4,4


In [31]:
user_activities_grouped['Судебное заседание'].sum(), user_activities_grouped['Запрос копий документов'].sum()

(8, 28)

Все сходится, значит все корректно преобразовано.

In [32]:
# добавлю итоговую таблицу кол-ва активностей по каждому дню

def count_all(row):
    cols_list = user_activities_grouped.columns
    count = 0
    for col in cols_list:
        count += row[col]
        
    return count

user_activities_grouped['Всего создано за день'] = user_activities_grouped.apply(count_all, axis=1)

In [33]:
user_activities_grouped.head(2)

,Судебное заседание,Судебное заседание СД 3Л,Передача дела в суд,Подготовка обращения в суд,Запрос на оплату госпошлины,Запрос расчета задолженности,Запрос копий документов,Всего создано за день
Дата создания испр,,,,,,,,
2022-11-21,0,3,5,0,0,0,0,8
2022-11-22,3,5,0,0,0,0,0,8


__Теперь  сформируем две таблицы - поступившие в работу новые дела и закрытые в проверяемом периоде:__

In [34]:
# ф-ция для объединения иной работы и ПЗ для послед. расчета поступивших новых дел и завершенных
def concatenate_data(data_main, data_other):
    """Соединение происходит по выбранным одинаковым названиям колонок в обоих выгрузках"""
    
    
    # в выгрузке иной работы поле Дата Создания наз-ся по другому - переименую его в поле Дата создания
    data_other.rename(columns={'Дата поступления' : 'Дата создания дела'}, inplace=True)
    
    # из обоих больших выгрузок всех дел оставляю только дела ответственного
    data_main = data_main.loc[data_main['user_id'] == user_to_check]
    data_other = data_other.loc[data_other['user_id'] == user_to_check]
    
    # общие колонки, имеющиеся в обеих выгрузках, берем лишь необходимые
    cols_list = ['Номер по SIEBEL', 'user_id', 'Дата создания дела', 'Дата закрытия', 'Статус']
    
    # в обоих выгрузках беру только те выбранные колонки
    data_main = data_main.loc[:, cols_list]
    data_other = data_other.loc[:, cols_list]
    
    # объединяю две выгрузки по колонкам
    common_data = pd.concat([data_main, data_other], sort=False, axis=0)
    print(f'Входные выгрузки имели кол-во строк: {data_main.shape[0]} и {data_other.shape[0]}')
    
    # сразу приведу формат даты в корректный тим
    common_data['Дата закрытия'] = common_data['Дата закрытия'].apply(lambda x: fix_date(x))
    common_data['Дата создания дела'] = common_data['Дата создания дела'].apply(lambda x: fix_date(x))
    
    return common_data

In [35]:
common_user_data = concatenate_data(data, data_other)

# смотрим кол-во строк в объединенном датасете
common_user_data.shape[0]

Входные выгрузки имели кол-во строк: 301 и 99


400

In [36]:
# смотрим что получилось
common_user_data

,Номер по SIEBEL,user_id,Дата создания дела,Дата закрытия,Статус
632,1-3287509057,100000023,2023-01-31,NaT,В работе
633,1-2562329755,100000023,2022-06-03,2022-10-13,Завершено
634,1-2562329815,100000023,2022-06-03,2022-07-25,Завершено
635,1-2566500500,100000023,2022-06-06,2022-10-24,Завершено
636,1-2839961896,100000023,2022-09-14,NaT,В работе
...,...,...,...,...,...
3191,1-197NYGD,100000023,2022-08-01,2023-01-16,Завершено
3192,1-1GKSPQM,100000023,2022-12-29,NaT,В работе
3202,1-1GFTQGY,100000023,2022-12-27,2023-02-10,Завершено
3206,1-16X45Q3,100000023,2022-06-21,NaT,В работе


__Выявляю закрытые дела в периоде и подготовливаю таблицу для дальнейшего соединения:__

In [37]:
closed_deals = common_user_data.loc[(common_user_data['Статус'] == 'Завершено') & 
                             ((common_user_data['Дата закрытия'] > start) & (common_user_data['Дата закрытия'] < end))]

closed_deals

,Номер по SIEBEL,user_id,Дата создания дела,Дата закрытия,Статус
689,1-2892622063,100000023,2022-09-28,2022-11-23,Завершено
835,1-2933131537,100000023,2022-10-14,2022-12-01,Завершено
836,1-2933131779,100000023,2022-10-14,2022-11-28,Завершено
840,1-2881536149,100000023,2022-09-27,2022-11-24,Завершено
920,1-2928316115,100000023,2022-10-13,2022-12-02,Завершено
931,1-2794925394,100000023,2022-08-30,2022-12-02,Завершено
2862,1-1EO07ZB,100000023,2022-11-14,2022-11-22,Завершено
2878,1-15H3F0N,100000023,2022-05-19,2022-11-22,Завершено
2965,1-191OUNF,100000023,2022-08-04,2022-11-21,Завершено


In [38]:
closed_deals_grouped = closed_deals.groupby('Дата закрытия').agg({'Статус': 'count'})
closed_deals_grouped.rename(columns={'Статус' : 'Кол-во закрытых дел в день'}, inplace=True)
closed_deals_grouped.reset_index(inplace=True)
closed_deals_grouped

,Дата закрытия,Кол-во закрытых дел в день
0,2022-11-21,1
1,2022-11-22,2
2,2022-11-23,1
3,2022-11-24,1
4,2022-11-28,1
5,2022-12-01,1
6,2022-12-02,2


__Выявляю новые дела в периоде и подготовливаю таблицу для дальнейшего соединения:__

In [39]:
new_deals = common_user_data.loc[(common_user_data['Дата создания дела'] > start) & (common_user_data['Дата создания дела'] < end)]

new_deals.shape

(31, 5)

In [40]:
new_deals_grouped = new_deals.groupby('Дата создания дела').agg({'Статус': 'count'})
new_deals_grouped.rename(columns={'Статус' : 'Кол-во новых дел в день'}, inplace=True)
new_deals_grouped.reset_index(inplace=True)
new_deals_grouped

,Дата создания дела,Кол-во новых дел в день
0,2022-11-21,4
1,2022-11-22,9
2,2022-11-23,3
3,2022-11-24,3
4,2022-11-28,3
5,2022-11-29,5
6,2022-11-30,2
7,2022-12-01,1
8,2022-12-02,1


In [41]:
# Дату создания у данной таблицы нужно вывести из индексов в колонку
user_activities_grouped.reset_index(inplace=True)

__Перед соденинением таблиц, смотрим на них еще раз:__

In [42]:
user_activities_grouped.head(2)

,Дата создания испр,Судебное заседание,Судебное заседание СД 3Л,Передача дела в суд,Подготовка обращения в суд,Запрос на оплату госпошлины,Запрос расчета задолженности,Запрос копий документов,Всего создано за день
0,2022-11-21,0,3,5,0,0,0,0,8
1,2022-11-22,3,5,0,0,0,0,0,8


In [43]:
closed_deals_grouped.head(2)

,Дата закрытия,Кол-во закрытых дел в день
0,2022-11-21,1
1,2022-11-22,2


In [44]:
new_deals.head(2)

,Номер по SIEBEL,user_id,Дата создания дела,Дата закрытия,Статус
657,1-3062082009,100000023,2022-11-22,2023-01-26,Завершено
681,1-3065680887,100000023,2022-11-23,2023-01-17,Завершено


In [45]:
court_hearings_table.head(2)

,Начало испр,Статус
0,2022-11-21,3
1,2022-11-22,6


In [46]:
# сразу поменяем название колонки
court_hearings_table.rename(columns={'Статус' : 'Кол-во запланир. с/з'}, inplace=True)

__Соединяю все 4 таблицы в одну:__

In [47]:
def merge_to_final_table(data1, data2, data3, data4):   # C ПЕРЕИМЕНОВАНИЯМИ ПОТОМ УПРОСТИТЬ
    """data1 - общая таблица активностей
       data2 - таблица закрытых
       data3 - таблица новых дел
       data4 - таблица запл. заседаний
    """
    # чтобы соединять по колонкам, они должны иметь одно названия
    data1.rename(columns={'Дата создания испр' : 'Дата'}, inplace=True)
    data2.rename(columns={'Дата закрытия' : 'Дата'}, inplace=True)
    data3.rename(columns={'Дата создания дела' : 'Дата'}, inplace=True)
    data4.rename(columns={'Начало испр' : 'Дата'}, inplace=True)
    
    common_table = pd.merge(data1, data2, on=('Дата'), how='outer')
    common_table = pd.merge(common_table, data3, on=('Дата'), how='outer')
    common_table = pd.merge(common_table, data4, on=('Дата'), how='outer')
    
    return common_table

In [48]:
result = merge_to_final_table(data1=user_activities_grouped, data2=closed_deals_grouped, data3=new_deals_grouped,\
                             data4=court_hearings_table)
result.fillna(0, inplace=True) 

In [49]:
result.sort_values(by='Дата', ascending=True)

,Дата,Судебное заседание,Судебное заседание СД 3Л,Передача дела в суд,Подготовка обращения в суд,Запрос на оплату госпошлины,Запрос расчета задолженности,Запрос копий документов,Всего создано за день,Кол-во закрытых дел в день,Кол-во новых дел в день,Кол-во запланир. с/з
0,2022-11-21,0.0,3.0,5.0,0.0,0.0,0.0,0.0,8.0,1.0,4.0,3.0
1,2022-11-22,3.0,5.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,9.0,6.0
2,2022-11-23,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,3.0,4.0
3,2022-11-24,1.0,0.0,9.0,9.0,9.0,9.0,9.0,46.0,1.0,3.0,2.0
4,2022-11-25,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0
5,2022-11-28,1.0,0.0,3.0,3.0,3.0,3.0,3.0,16.0,1.0,3.0,5.0
6,2022-11-29,1.0,0.0,6.0,6.0,8.0,6.0,6.0,33.0,0.0,5.0,1.0
7,2022-11-30,2.0,4.0,4.0,4.0,4.0,4.0,4.0,26.0,0.0,2.0,6.0
9,2022-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0
8,2022-12-02,0.0,0.0,5.0,5.0,4.0,4.0,4.0,22.0,2.0,1.0,0.0


In [50]:
writer = pd.ExcelWriter(f'Аналитика активности {user_to_check} за период с {str(start)} по {str(end)}.xlsx')
result.to_excel(writer) 
writer.save()